# Homework #10:
|Author| Stanley A. Baronett|
|--|-------------------------------|
|Created | 11/2/2019|
|Updated | 11/3/2019|

## 17.5 Global Optimization I
We wish to find the ground state energy between three neutral atoms or molecules, $N=3$. To simplify the calculation, we'll use the _Lennard-Jones (LJ) potential_:

$$ V = 4\varepsilon \left[ \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^{6} \right] \tag{1},$$

where $\varepsilon$ is the depth of the potential well, $\sigma$ is the finite distance at which the interparticle potential is zero, and $r$ is the distance between the particles. We will also assume $\varepsilon$ = $\sigma$ = 1.

In [1]:
def LJ(r):
    """
    Calculate and return the Lennard-Jones potential between two
    atoms as a function of separation distance.
    Input:
        r: interatomic separation distance.
    Output:
        Lennard-Jones potential with ε = σ = 1
    """
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

Since $N=3$, the number of interatomic pairs will be

$$ \frac{N\times(N-1)}{2} = 3 \tag{2}.$$

Thus, we'll need to calculate the distance $r$ for each pair and evaluate its corresponding LJ$_3$ potential.

In [2]:
import numpy as np

def total_LJ_V(positions):
    """
    Calculate the total potential energy of N number of atoms using
    the Lennard-Jones potential.
    Input:
        positions: 3*N 1-D array which represents the atomic positions
                   (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    Output:
        E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    for i in range(N_atom - 1):
        for j in range(i + 1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1 - pos2)
            E += LJ(dist)
            
    return E

Now that we have a way to calculate the total energy of $N$ atoms, using the LJ potential, next we need to randomly generate positions for these $N$ atoms. To do so, we'll generate a random sample for the $x, y,$ and $z$ coordinates of each atom to be within some cubic box with set length $L$.

In [145]:
def init_pos(N, L=1):
    """
    Randomly initialize the 3-D position of N atoms within a cube
    of length L.
    Input:
        N: the number of atoms
        L: 3-D boundary of random sample (e.g., length of cube)
    Output:
        3*N 1-D array of atomic positions
        (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    """
    return L*np.random.random_sample((N*3,))

Before we attempt to find the _gound state_ of the LJ$_3$ potential for three atoms, let's go ahead and generate a random configuration of three atoms and visualize it using the ___Atomic Simulation Environment___ (ASE) library.

In [173]:
# Visulization of the LJ clusters
from ase.visualize import view
from ase import Atoms

pos = init_pos(3)
N = 3
cluster = Atoms('N'+str(N), positions=np.reshape(pos*2.0,[N,3]))
view(cluster, viewer='x3d')

Of course, the configuration isn't symmetric. Nonetheless, let's calculate it's LJ potential energy.

In [174]:
print("Total energy: ", total_LJ_V(pos))

7102462.193920587

As we might have expected, this configuration corresponds to a high (positive) potential energy, and definitely does not represent the ground state configuration for three atoms.

Now let's see if we can _minimize_ the total energy _function_ to determine a minimum, hopefully corresponding to the ground state configuration (the global minimum).

In [175]:
from scipy.optimize import minimize

res = minimize(total_LJ_V, pos, method='CG', tol=1e-9)
print("Total energy: ", res.fun)

-3.0


With a single minimization pass, using the randomly generated spatial configuration, we found a ground state energy of -3.0. For reference, the energy value tabulated at the Cambridge LJ cluster database ([link](http://doye.chem.ox.ac.uk/jon/structures/LJ/tables.150.html)) for $N=3$ is "-3.000000." Not bad.

Now let's visualize what this ground state configuration looks like.

In [176]:
ground = res.x # extract the spatial coordinates of the three atoms
               # corresponding to the minimum (ground state)
N = 3
cluster = Atoms('N'+str(N), positions=np.reshape(ground*2.0,[N,3]))
view(cluster, viewer='x3d')

We now see the expected symmetry that was missing in the earlier, purely random, configuration. The ground state for $N=3$ corresponds to each atom at the point of an _equilateral triangle_.

## 18.3 Global Optimization II

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show 
1. the average number of attempts to find the ground state
2. the time costs


### Optional
try to improve the code to make it run faster, analyze the most time consuming part and give your solution


In [ ]:
def LJ_jac(r):
    """
    Calculate and return the Jacobian of the Lennard-Jones potential
    between two atoms as a function of separation distance.
    Input:
        r: interatomic separation distance.
    Output:
        Lennard-Jones potential with ε = σ = 1
    """
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

In [ ]:
def total_LJ_V_jac(positions):
    """
    Calculate the Jacobian of the total potential energy 
    of N number of atoms using the Lennard-Jones potential.
    Input:
        positions: 3*N 1-D array which represents the atomic positions
                   (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    Output:
        E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    for i in range(N_atom - 1):
        for j in range(i + 1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1 - pos2)
            E += LJ_jac(dist)
            
    return E